In [39]:
# 데이터 가져오기
import numpy as np
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/datasets/titanic/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/datasets/titanic/test.csv')

In [40]:
# 앞 데이터 보기1
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
# 앞 데이터 보기2
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [42]:
# 열 정보 보기1
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [43]:
# 열 정보 보기2
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [44]:
# Null 건수 보기1
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [45]:
# Null 건수 보기2
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [46]:
train_df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [47]:
train_df['Cabin'].value_counts()

C23 C25 C27    4
B96 B98        4
G6             4
F2             3
D              3
              ..
A24            1
A14            1
B79            1
C30            1
B73            1
Name: Cabin, Length: 147, dtype: int64

In [48]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [49]:
# Null 처리 함수
def fillna(df):
  df['Age'].fillna(df['Age'].median(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

In [50]:
# 불필요한 속성 제거
def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
  return df

In [51]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코딩 수행
def encoding_features(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin', 'Sex', 'Embarked']
  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
  return df

In [67]:
# 데이터 전처리 함수 호출
def preprocessing_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = encoding_features(df)
  return df

In [53]:
X_train_all = train_df.drop('Survived', axis=1)
y_train_all = train_df['Survived']

In [54]:
X_train_all = preprocessing_features(X_train_all)
X_train_all.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,7,3
1,1,0,38.0,1,0,71.2833,2,0
2,3,0,26.0,0,0,7.9250,7,3
3,1,0,35.0,1,0,53.1000,2,3
4,3,1,35.0,0,0,8.0500,7,3


In [55]:
# 데이터 분리하기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=11)
X_train.shape, X_test.shape

((712, 8), (179, 8))

In [57]:
# 사이킷런 알고리즘 import하기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [58]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print(f'DecisionTreeClassifier 정확도 : {accuracy_score(y_test, dt_pred)}')

rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print(f'RandomForestClassifier 정확도 : {accuracy_score(y_test, rf_pred)}')

lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print(f'LogisticRegression 정확도 : {accuracy_score(y_test, lr_pred)}')

DecisionTreeClassifier 정확도 : 0.8212290502793296
RandomForestClassifier 정확도 : 0.8547486033519553
LogisticRegression 정확도 : 0.8491620111731844


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [60]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf_clf, X_train_all, y_train_all, cv=5)
for iter_count, accuracy in enumerate(scores):
  print(f'교차검증 정확도 #{iter_count} : {accuracy}')

print(f'평균 정확도 : {np.mean(scores)}')

교차검증 정확도 #0 : 0.7988826815642458
교차검증 정확도 #1 : 0.7921348314606742
교차검증 정확도 #2 : 0.8426966292134831
교차검증 정확도 #3 : 0.7584269662921348
교차검증 정확도 #4 : 0.8426966292134831
평균 정확도 : 0.8069675475488042


In [61]:
# Decision Tree
params = {
    'max_depth': [2, 3, 5, 10],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 5, 8]
}

gs1 = GridSearchCV(dt_clf, param_grid=params, n_jobs=-1)

gs1.fit(X_train, y_train)

print('best parameter:', gs1.best_params_)
print('best score:', gs1.best_score_)
print(gs1.score(X_train, y_train))
print(gs1.score(X_test, y_test))

best parameter: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
best score: 0.7991825076332119
0.8117977528089888
0.8715083798882681


In [62]:
# Random Forest
gs2 = GridSearchCV(rf_clf, param_grid=params, n_jobs=-1)

gs2.fit(X_train, y_train)
 
print('best parameter:', gs2.best_params_)
print('best score:', gs2.best_score_)
print(gs2.score(X_train, y_train))
print(gs2.score(X_test, y_test))

best parameter: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}
best score: 0.8132079188417217
0.8553370786516854
0.8659217877094972


In [63]:
# Logistic Regression
params = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 5, 10],
    'max_iter': [500, 750, 1000]
}

gs3 = GridSearchCV(lr_clf, param_grid=params, n_jobs=-1)

gs3.fit(X_train, y_train)

print('best parameter:', gs3.best_params_)
print('best score:', gs3.best_score_)
print(gs3.score(X_train, y_train))
print(gs3.score(X_test, y_test))

best parameter: {'C': 0.1, 'max_iter': 500, 'penalty': 'l2'}
best score: 0.7794838963853049
0.7808988764044944
0.8547486033519553


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solve

In [68]:
test_df = preprocessing_features(test_df)
test_df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,34.5,0,0,7.8292,7,1
1,3,0,47.0,1,0,7.0000,7,2
2,2,1,62.0,0,0,9.6875,7,1


In [69]:
# test.csv : 2차원 배열로 변환하기
pred_input = test_df.values
pred_input

array([[ 3.    ,  1.    , 34.5   , ...,  7.8292,  7.    ,  1.    ],
       [ 3.    ,  0.    , 47.    , ...,  7.    ,  7.    ,  2.    ],
       [ 2.    ,  1.    , 62.    , ...,  9.6875,  7.    ,  1.    ],
       ...,
       [ 3.    ,  1.    , 38.5   , ...,  7.25  ,  7.    ,  2.    ],
       [ 3.    ,  1.    , 27.    , ...,  8.05  ,  7.    ,  2.    ],
       [ 3.    ,  1.    , 27.    , ..., 22.3583,  7.    ,  0.    ]])

In [70]:
# 모델 예측값 얻기
best_dt = gs1.best_estimator_
pred_dt = best_dt.predict(pred_input)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [72]:
# 파일 저장하기
output = pd.DataFrame({
    'id': test_df.index,
    'Survived': pred_dt})

output.to_csv('/content/drive/MyDrive/datasets/titanic/my_submission.csv', index=False)